In [1]:
import pandas as pd
import json
import pprint
import ast
import gzip
from itertools import chain
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [2]:
#json.loads('australian_user_reviews.json')

In [3]:
# with open('australian_user_reviews.json') as f:
#     s = f.read()
# f.close()

In [4]:
# s

In [5]:
#l = eval(s)

In [6]:
#json_string = json.dumps(s)

In [7]:
#json_loads = json.loads(json_string)

In [8]:
#json_loads

In [9]:
# f = open('australian_user_reviews-Copy1.txt')
# s = f.read().strip()
# l = ast.literal_eval(s)

In [10]:
#f

In [11]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [12]:
data_test = parse('australian_user_reviews.json.gz')

In [13]:
data_test

<generator object parse at 0x7fcbb6d01e40>

In [14]:
full_data = parse('steam_reviews.json.gz')

In [15]:
full_data

<generator object parse at 0x7fcbb6d013c0>

In [16]:
game_data = parse('steam_games.json.gz')

In [17]:
game_data

<generator object parse at 0x7fcbb6d13120>

In [18]:
item_data = parse('australian_users_items.json.gz')

In [19]:
test_df = pd.DataFrame(data_test)

In [20]:
# df = pd.DataFrame(full_data)

In [21]:
game_df = pd.DataFrame(game_data)

In [22]:
item_df = pd.DataFrame(item_data)

In [23]:
item_df

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [24]:
test_df['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [25]:
# item_df['items'][1]

In [26]:
game_ids = [ids for ids in item_df['items'][0][0]['item_id']]

In [27]:
game_ids

['1', '0']

In [28]:
# for ids in range(0,len(game_ids)):
#     int(item_df['items'][0][ids]['item_id'])
#     for game_id in int(item_df['items'][0][ids]['item_id']:
#         all_game_id.append(game_id)

In [29]:
#This saves all of the game ids a user has into a 2d list - this will be used to save the info for the rec system
all_game_id = []
for ids in item_df['items']:
    temp_list = []
    for count in range(len(ids)):
        #print(ids[count]['item_id'])
        temp_list.append(int(ids[count]['item_id']))
    #print(temp_list)
    all_game_id.append(temp_list)
    #print(len(ids))

In [30]:
len(all_game_id[1])

888

In [31]:
item_df['game_ids'] = all_game_id

In [32]:
item_df.head()

,user_id,items_count,steam_id,user_url,items,game_ids
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."


In [33]:
flat_game_ids = set(chain.from_iterable(all_game_id))

In [34]:
list_flat_games = sorted(list(flat_game_ids))

In [35]:
len(list_flat_games)

10978

In [36]:
testing_df = item_df.copy()

In [37]:
testing_df['game_ids'][0][1]

20

In [38]:
item_df

,user_id,items_count,steam_id,user_url,items,game_ids
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."
...,...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla...","[413850, 413851, 413852, 413853, 413854, 41385..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania...","[11020, 6370, 13140, 51100, 107900, 113400, 20..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[],[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'...","[304930, 227940, 346330, 373330, 388490, 52157..."


In [39]:
# This cell makes a column for each game and applies a zero. 
# then would go through and change value to a 1 if the game id was present in the list. However, I think this is not 
# How I want to tackle this but instead make a bunch more rows 


# count = 0;
# for game in list_flat_games:
#     testing_df[f'game_{game}'] = 0
# #     if game in testing_df['game_ids'][count]:
# #         testing_df[f'game_{game}'][count]

In [40]:
item_df['ids'] = item_df.index

In [41]:
item_df

,user_id,items_count,steam_id,user_url,items,game_ids,ids
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...",0
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...",1
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...",2
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...",3
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...",4
...,...,...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla...","[413850, 413851, 413852, 413853, 413854, 41385...",88305
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania...","[11020, 6370, 13140, 51100, 107900, 113400, 20...",88306
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[],[],88307
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'...","[304930, 227940, 346330, 373330, 388490, 52157...",88308


In [42]:
# df['user_id'].value_counts()

In [43]:
# df['username'].value_counts()

In [44]:
game_df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [45]:
# game_df.loc[game_df['id'] == 725280]

This makes it so the game id explodes and saves the user id to the game they have played

In [46]:
clean_df = item_df[['ids','game_ids']]

In [47]:
clean_df = clean_df.explode('game_ids')

In [48]:
clean_df.reset_index(inplace=True,drop=True)

In [49]:
clean_df

,ids,game_ids
0,0,10
1,0,20
2,0,30
3,0,40
4,0,50
...,...,...
5170010,88308,373330
5170011,88308,388490
5170012,88308,521570
5170013,88308,519140


In [50]:
clean_df['game_ids'].isna().sum()

16806

In [51]:
clean_df['owned'] = 1

In [52]:
clean_df

,ids,game_ids,owned
0,0,10,1
1,0,20,1
2,0,30,1
3,0,40,1
4,0,50,1
...,...,...,...
5170010,88308,373330,1
5170011,88308,388490,1
5170012,88308,521570,1
5170013,88308,519140,1


In [53]:
owned = clean_df.pivot_table(index=['ids'],columns=['game_ids'],values='owned')

In [54]:
owned

game_ids,10,20,30,40,50,60,70,80,100,130,...,527520,527570,527810,527890,527900,528580,528660,529670,529820,530720
ids,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
owned = owned.dropna(thresh=10,axis=1).fillna(0)

In [56]:
owned

game_ids,10,20,30,40,50,60,70,80,100,130,...,523485,523486,523487,523488,523489,523490,523790,524030,527340,527810
ids,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# similar_games_df = owned.corr(method='pearson')

In [58]:
# similar_games_df.head(50)

In [59]:
clean_df

,ids,game_ids,owned
0,0,10,1
1,0,20,1
2,0,30,1
3,0,40,1
4,0,50,1
...,...,...,...
5170010,88308,373330,1
5170011,88308,388490,1
5170012,88308,521570,1
5170013,88308,519140,1


In [60]:
reader = Reader()
data = Dataset.load_from_df(clean_df,reader)

In [61]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  88310 

Number of items:  10979


In [62]:
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [63]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.0011264635395874956, 'mae': 0.00016516277797406646}
{'rmse': {'n_factors': 20, 'reg_all': 0.1}, 'mae': {'n_factors': 20, 'reg_all': 0.1}}


In [ ]:
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [ ]:
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))